<h2><b>0) install PyRasgo with extra [df] dependencies</b></h2>

In [ ]:
!pip install pyrasgo[df] --upgrade

In [18]:
import pyrasgo
import pandas as pd
import numpy as np

<h2><b>1) Register for Rasgo or copy your api key from the <a href="https://app.rasgoml.com/">Rasgo UI</a> and paste below</b></h2>

In [5]:
#paste your API key here if you have one already
api_key=''
rasgo = pyrasgo.connect(api_key)

In [ ]:
#otherwise, register for a free Rasgo API Key
pyrasgo.register(email="fred@flinstone.com", password="")

#login and pyrasgo will retrieve your API key for you
rasgo = pyrasgo.login(email="fred@flinstone.com", password="")

<h2><b>2) get a dataframe from somewhere :) </b></h2>

In [46]:
df = pd.read_csv('sample-sales-data.csv')#, parse_dates=['DATE'])
df

,DATE,FIPS,DS_TEMP_VARIATION_PREVIOUS_DAY,DS_DAILY_CLOUDCOVER,DS_WEATHER_ICON,DS_DAILY_LOW_TEMP,DS_DAILY_HIGH_TEMP,DS_DAILY_TOTAL_RAINFALL,STORE_BRAND_ID,TOTAL_TRAFFIC,DINEIN_TRAFFIC,DRIVETHRU_TRAFFIC
0,11/21/20,6077,-1.12,0.18,clear-day,32.24,62.27,0.00003,3-1100801,258,9,218
1,11/21/20,6077,-1.12,0.18,clear-day,32.24,62.27,0.00003,3-1100308,311,4,238
2,11/22/20,6077,3.26,0.26,clear-day,32.62,65.53,0.00005,3-1100801,225,7,197
3,11/22/20,6077,3.26,0.26,clear-day,32.62,65.53,0.00005,3-1100308,246,0,186
4,11/27/20,6077,-2.07,0.05,clear-day,32.25,63.90,0.00002,3-1100801,227,5,189
...,...,...,...,...,...,...,...,...,...,...,...,...
24654,3/2/20,53033,3.23,0.91,rain,30.88,40.62,0.03516,3-1101688,203,32,144
24655,3/2/20,53033,3.23,0.91,rain,30.88,40.62,0.03516,Jan-72,197,35,96
24656,3/2/20,53033,3.23,0.91,rain,30.88,40.62,0.03516,Jan-71,267,42,121
24657,3/2/20,53033,3.23,0.91,rain,30.88,40.62,0.03516,1-1627,167,80,0


<h2><b>3) pick a target feature</b></h2>

In [47]:
target = 'TOTAL_TRAFFIC'

#exclude both dimensions and leakage features
exclude_list = ['STORE_BRAND_ID', 'DATE', 'FIPS','DINEIN_TRAFFIC','DRIVETHRU_TRAFFIC']

<h2><b>4) profile & evaluate your features!</b></h2>

In [48]:
r = rasgo.evaluate.profile(df)

In [22]:
r = rasgo.evaluate.feature_importance(df, target_column=target, exclude_columns=exclude_list)

<h2><b>5) create some new features & re-evaluate!</b></h2>


In [23]:
#some lag features
df['7DAYLAG_TRAFFIC'] = df.groupby(['DATE','FIPS'])['TOTAL_TRAFFIC'].shift(7)
df['1DAYLAG_TRAFFIC'] = df.groupby(['DATE','FIPS'])['TOTAL_TRAFFIC'].shift(1)

df['7DAYLAG_DRIVETHRU'] = df.groupby(['DATE','FIPS'])['DRIVETHRU_TRAFFIC'].shift(7)
df['1DAYLAG_DRIVETHRU'] = df.groupby(['DATE','FIPS'])['DRIVETHRU_TRAFFIC'].shift(1)

In [24]:
r = rasgo.evaluate.feature_importance(df, target_column=target, exclude_columns=exclude_list)

In [29]:
#ratio and moving avg features
df['YESTERDAY_PERCENTDRIVETHRU'] = df['1DAYLAG_DRIVETHRU']/df['1DAYLAG_TRAFFIC']
df['7DayMovingAvgTraffic'] = df.groupby(['DATE','FIPS'])['TOTAL_TRAFFIC'].transform(lambda x: x.rolling(7, 1, closed='neither').mean())

In [30]:
r = rasgo.evaluate.feature_importance(df, target_column=target, exclude_columns=exclude_list)

In [44]:
#some more features
#df['WEEKDAY'] = np.where((df['DATE']).dt.dayofweek < 5,0,1)
df['RainBin'] = np.where(df['DS_DAILY_TOTAL_RAINFALL']>.02, 'Rain', 'No Rain')

In [49]:
r = rasgo.evaluate.feature_importance(df, target_column=target, exclude_columns=exclude_list)

<h2><b>6) prune your features and create your train and test sets</b></h2>

In [53]:
df = rasgo.prune.features(df, target_column=target, pct_of_top_feature=.05, exclude_columns=exclude_list)

Prune Method: Keeping features with importance above 0.1 pct of max feature
Dropped features below importance threshold 1.472474403435804: []              


In [16]:
r = rasgo.evaluate.feature_importance(df, target_column=target, exclude_columns=exclude_list)

In [51]:
train_df, backtest_df = rasgo.evaluate.train_test_split(df, training_percentage=.8, timeseries_index='DATE')